# Trend analysis: Check significant increasing of summer temperature with ERA5-Land monthly aggregated data

## Import packages

In [ ]:
import calendar

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pymannkendall as mk
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Set variables for request

In [ ]:
# Time
year_start = 1997
year_stop = 1998

## Set the data request

In [ ]:
collection_id = "reanalysis-era5-land-monthly-means"
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "time": "00:00",
    "area": [44, -10, 36, 0],
}

## Download data

In [ ]:
ds = download.download_and_transform(collection_id, request, chunks={"year": 1})

## Plot annual mean

In [ ]:
ds_annual = diagnostics.time_weighted_mean(ds)
plot.projected_map(
    ds_annual["t2m"].where(ds_annual["t2m"]), projection=ccrs.PlateCarree()
)

## Plot spatial mean in JAS

In [ ]:
season_months = [7, 8, 9]
ds_spatial = diagnostics.spatial_weighted_mean(ds)
da = ds_spatial["t2m"].where(
    ds_spatial["forecast_reference_time"].dt.month.isin(season_months), drop=True
)
da.plot()
plt.title(", ".join([calendar.month_name[month] for month in season_months]))

trend, h, p, z, tau, s, var_s, slope, intercept = mk.original_test(da)
is_significant = p < 0.05
print(f"The trend is{'' if is_significant else ' NOT'} significant.")
print(f"Trend: {slope}")